In [1]:
import rasterio
import numpy as np

def read_raster(file_path):
    with rasterio.open(file_path) as dataset:
        data = dataset.read(1)
    return data

def compare_forest_change(first_map, second_map):
    loss = (first_map == 6) & (second_map != 6)
    gain = (first_map != 6) & (second_map == 6)
    return loss, gain

def calculate_area(pixels, pixel_size=400):  # 20m x 20m = 400m**2 per pixel
    return np.sum(pixels) * pixel_size

def create_change_map(first_path, second_path, output_path, txt_output_path):
    first_map = read_raster(first_path)
    second_map = read_raster(second_path)

    loss, gain = compare_forest_change(first_map, second_map)

    change_map = np.zeros(first_map.shape, dtype=np.int8)
    change_map[loss] = 1
    change_map[gain] = 2

    loss_area = calculate_area(loss)
    gain_area = calculate_area(gain)
    total_change = gain_area - loss_area
    
    print(f"Number of pixels with forest loss: {np.sum(change_map == 1)}")
    print(f"Number of pixels with forest gain: {np.sum(change_map == 2)}")

    with open(txt_output_path, 'w') as txt_file:
        txt_file.write(f"Total area of loss: {loss_area} m^2\n")
        txt_file.write(f"Total area of gain: {gain_area} m^2\n")
        txt_file.write(f"Total change: {total_change} m^2\n")

    with rasterio.open(first_path) as src:
        profile = src.profile
        profile.update(dtype=rasterio.int8, count=1)

        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(change_map, 1)

first_file =''
second_file = ''
output_file = ''
txt_output_file = ''

create_change_map(first_file, second_file, output_file, txt_output_file)

C:\Users\karo1\AppData\Local\Temp\ipykernel_23876\2836274287.py:15: RuntimeWarning: overflow encountered in long_scalars
  return np.sum(pixels) * pixel_size
C:\Users\karo1\AppData\Local\Temp\ipykernel_23876\2836274287.py:29: RuntimeWarning: overflow encountered in long_scalars
  total_change = gain_area - loss_area


Number of pixels with forest loss: 2497300
Number of pixels with forest gain: 7389276
